In [3]:
import pandas as pd
from transformers import pipeline
import numpy as np


def decompose_sentences(text):
    # 실제로는 GPT-3.5를 사용하여 복잡한 문장을 단일 주장 문장으로 분해합니다.
    # 여기서는 간단히 문장을 분할하는 데모를 사용합니다.
    
    return text.split('.')

# NLI 모델 로드
nli_model = pipeline("text-classification", model="roberta-large-mnli")

# 문장 간 논리적 관계 평가
def nli_score(sentence1, sentence2):
    result1 = nli_model(f"{sentence1} entails {sentence2}")
    result2 = nli_model(f"{sentence2} entails {sentence1}")
    
    label1 = result1[0]['label']
    label2 = result2[0]['label']
    
    if label1 == 'CONTRADICTION' and label2 == 'ENTAILMENT':
        return 'neutral'
    elif label1 == 'ENTAILMENT' and label2 == 'CONTRADICTION':
        return 'neutral'
    elif label1 == 'CONTRADICTION' or label2 == 'CONTRADICTION':
        return 'contradiction'
    elif label1 == 'ENTAILMENT' or label2 == 'ENTAILMENT':
        return 'entailment'
    else:
        return 'neutral'

# CASPR 점수 계산
def calculate_caspr(summary1, summary2):
    sentences1 = decompose_sentences(summary1)
    print(sentences1)
    sentences2 = decompose_sentences(summary2)
    print(sentences2)
    
    scores = []
    for s1 in sentences1:
        for s2 in sentences2:
            relationship = nli_score(s1, s2)
            if relationship == 'contradiction':
                scores.append(1)
            elif relationship == 'entailment':
                scores.append(-1)
            else:
                scores.append(0)
    
    if not scores:
        return 0
    
    return (sum(scores) / len(scores) + 1) /2 * 100

# 샘플 데이터
summary_A_B = "The hotel is clean. The breakfast is expensive."
summary_B_A = "The hotel is not clean. The breakfast is not cheap." # 100

# summary_A_B = "The hotel is sparkly clean."
# summary_B_A = "The hotel was kept very tidy." # 0이 나와야하는데 50이 나온다..

# CASPR 점수 계산
caspr_score = calculate_caspr(summary_A_B, summary_B_A)
print(f"CASPR Score: {caspr_score}")


Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['The hotel is clean', ' The breakfast is expensive', '']
['The hotel is not clean', ' The breakfast is not cheap', '']
CASPR Score: 88.88888888888889


In [8]:
import pandas as pd
from transformers import pipeline
import numpy as np


def decompose_sentences(text):
    # 실제로는 GPT-3.5를 사용하여 복잡한 문장을 단일 주장 문장으로 분해합니다.
    # 여기서는 간단히 문장을 분할하는 데모를 사용합니다.
    sentences = text.split('.')
    print(f"\nDecomposed Sentences: {sentences}\n")
    return sentences

# NLI 모델 로드
nli_model = pipeline("text-classification", model="roberta-large-mnli")

# 문장 간 논리적 관계 평가
def nli_score(sentence1, sentence2):
    result1 = nli_model(f"{sentence1.strip()} entails {sentence2.strip()}")
    result2 = nli_model(f"{sentence2.strip()} entails {sentence1.strip()}\n")
    
    label1 = result1[0]['label']
    label2 = result2[0]['label']
    
    print(f"Comparing '{sentence1}' and '{sentence2}':")
    print(f" - Result 1: {result1}")
    print(f" - Result 2: {result2}\n")
    
    if label1 == 'CONTRADICTION' and label2 == 'ENTAILMENT':
        return 'neutral'
    elif label1 == 'ENTAILMENT' and label2 == 'CONTRADICTION':
        return 'neutral'
    elif label1 == 'CONTRADICTION' or label2 == 'CONTRADICTION':
        return 'contradiction'
    elif label1 == 'ENTAILMENT' or label2 == 'ENTAILMENT':
        return 'entailment'
    else:
        return 'neutral'

# CASPR 점수 계산
def calculate_caspr(summary1, summary2):
    sentences1 = decompose_sentences(summary1)
    print(f"Sentences from Summary 1: {sentences1}")
    sentences2 = decompose_sentences(summary2)
    print(f"Sentences from Summary 2: {sentences2}\n")
    
    scores = []
    for s1 in sentences1:
        if not s1.strip():
            continue
        for s2 in sentences2:
            if not s2.strip():
                continue
            relationship = nli_score(s1, s2)
            print(f"Relationship between '{s1}' and '{s2}': {relationship}\n")
            if relationship == 'contradiction':
                scores.append(1)
            elif relationship == 'entailment':
                scores.append(-1)
            else:
                scores.append(0)
        print(f"for문 1 _ Scores: {scores}\n")
    
    if not scores:
        return 0
    
    print(f"Scores: {scores}\n")
    return (sum(scores) / len(scores) + 1)  * 100

# 샘플 데이터
summary_A_B = "The hotel is clean. The breakfast is expensive."
summary_B_A = "The hotel is not clean. The breakfast is not cheap."

# CASPR 점수 계산
caspr_score = calculate_caspr(summary_A_B, summary_B_A)
print(f"CASPR Score: {caspr_score}\n\n\n")

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



Decomposed Sentences: ['The hotel is clean', ' The breakfast is expensive', '']

Sentences from Summary 1: ['The hotel is clean', ' The breakfast is expensive', '']

Decomposed Sentences: ['The hotel is not clean', ' The breakfast is not cheap', '']

Sentences from Summary 2: ['The hotel is not clean', ' The breakfast is not cheap', '']

Comparing 'The hotel is clean' and 'The hotel is not clean':
 - Result 1: [{'label': 'CONTRADICTION', 'score': 0.9935221076011658}]
 - Result 2: [{'label': 'CONTRADICTION', 'score': 0.994443953037262}]

Relationship between 'The hotel is clean' and 'The hotel is not clean': contradiction

Comparing 'The hotel is clean' and ' The breakfast is not cheap':
 - Result 1: [{'label': 'NEUTRAL', 'score': 0.9528499245643616}]
 - Result 2: [{'label': 'NEUTRAL', 'score': 0.8089044690132141}]

Relationship between 'The hotel is clean' and ' The breakfast is not cheap': neutral

for문 1 _ Scores: [1, 0]

Comparing ' The breakfast is expensive' and 'The hotel is not

# 코드 수정 버전 

In [2]:
import pandas as pd
from transformers import pipeline
import numpy as np


def decompose_sentences(text):
    # 실제로는 GPT-3.5를 사용하여 복잡한 문장을 단일 주장 문장으로 분해합니다.
    # 여기서는 간단히 문장을 분할하는 데모를 사용합니다.
    sentences = text.split('.')
    print(f"Decomposed Sentences: {sentences}\n")
    return sentences

# NLI 모델 로드
nli_model = pipeline("text-classification", model="roberta-large-mnli")

# 문장 간 논리적 관계 평가
def nli_score(sentence1, sentence2):
    result1 = nli_model(f"{sentence1.strip()} entails {sentence2.strip()}")
    result2 = nli_model(f"{sentence2.strip()} entails {sentence1.strip()}\n")
    
    label1 = result1[0]['label']
    label2 = result2[0]['label']
    
    print(f"Comparing '{sentence1}' and '{sentence2}':")
    print(f" - Result 1: {result1}")
    print(f" - Result 2: {result2}\n")
    
    if label1 == 'CONTRADICTION' and label2 == 'ENTAILMENT':
        return 'neutral'
    elif label1 == 'ENTAILMENT' and label2 == 'CONTRADICTION':
        return 'neutral'
    elif label1 == 'CONTRADICTION' or label2 == 'CONTRADICTION':
        return 'contradiction'
    elif label1 == 'ENTAILMENT' or label2 == 'ENTAILMENT':
        return 'entailment'
    else:
        return 'neutral'

# CASPR 점수 계산
def calculate_caspr(summary1, summary2):
    flag = True # 첫 번째가 summary1, 두 번째가 summary2
    scores = []
    for i in range(0,2):
        if flag == True:
            sentences1 = decompose_sentences(summary1)
            print(f"Sentences from Summary 1: {sentences1}")
            sentences2 = decompose_sentences(summary2)
            print(f"Sentences from Summary 2: {sentences2}\n")
        else: # 첫 번째가 summary2, 두 번째가 summary1
            sentences1 = decompose_sentences(summary2)
            print(f"Sentences from Summary 1: {sentences1}")
            sentences2 = decompose_sentences(summary1)
            print(f"Sentences from Summary 2: {sentences2}\n")
    
                
        for s1 in sentences1:
            contradictions = 0
            entailments = 0 
            neutrals = 0
            if not s1.strip():
                continue
            for s2 in sentences2:
                if not s2.strip():
                    continue
                relationship = nli_score(s1, s2)
                print(f"Relationship between '{s1}' and '{s2}': {relationship}\n")
                if relationship == 'contradiction':
                    contradictions += 1
                    # scores.append(1)
                elif relationship == 'entailment':
                    entailments += 1
                    # scores.append(-1)
                else:
                    neutrals += 1
                    # scores.append(0)
                    
            # 개수 세서 점수 매기기 
            if contradictions == 0 and entailments == 0: # neutrals == len(sentences2)
                # 모든 문장들이 다 Neutral이므로, +1
                scores.append(1)
            elif contradictions > entailments:
                scores.append(1)
            elif contradictions <= entailments:
                scores.append(-1)
                
            print(f"s1 : {s1}\n")
            print(f"Contradictions: {contradictions}, Entailments: {entailments}, Neutrals: {neutrals}")

        flag = False
        
    if not scores:
        return 0
    
    print(f"Scores: {scores}\n")
    
    caspr_score = (sum(scores) / len(scores) + 1) * 50  # Adjusted to range [0, 100]
    return caspr_score

# 샘플 데이터 1
summary_A_B_1 = "The hotel is sparkly clean."
summary_B_A_1 = "The hotel was kept very tidy."
# summary_A_B_1 = "The hotel has a great location. The staff were friendly and helpful. The rooms were spacious and clean."
# summary_B_A_1 = "The hotel is situated in a prime location. The employees were kind and accommodating. The room was large and tidy."

# # 샘플 데이터 2
summary_A_B_2 = "The hotel is clean"
summary_B_A_2 = "The hotel is not clean"
# summary_A_B_2 = "The breakfast was delicious but expensive. The hotel amenities were top-notch. The view from the room was breathtaking."
# summary_B_A_2 = "The morning meal was tasty yet pricey. The hotel's facilities were excellent. The room offered a stunning view."

# # 샘플 데이터 3
# summary_A_B_3 = "The hotel room was dirty and the service was slow. The pool was overcrowded. I would not recommend this hotel."
# summary_B_A_3 = "The cleanliness of the room was poor and the service was delayed. The swimming pool was too crowded. I do not recommend this hotel."

# CASPR 점수 계산
# print("Sample Data 1:")
# caspr_score_1 = calculate_caspr(summary_A_B_1, summary_B_A_1)
# print(f"CASPR Score: {caspr_score_1}\n")

# print("\nSample Data 2:")
# caspr_score_2 = calculate_caspr(summary_A_B_2, summary_B_A_2)
# print(f"CASPR Score: {caspr_score_2}")

# print("\nSample Data 3:")
# caspr_score_3 = calculate_caspr(summary_A_B_3, summary_B_A_3)
# print(f"CASPR Score: {caspr_score_3}")

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [18]:
# 데이터셋 불러와서 실험하기
import json

# Load the JSON data
with open('/root/pretty_anno.json', 'r') as file:
    data = json.load(file)

# Initialize empty lists to store the first summaries
entity_a_summaries = []
entity_b_summaries = []

# Iterate through each item in the train array
for item in data['train']:
    # Extract the first element from entity_a_summary and entity_b_summary
    entity_a_first_summary = item['entity_a_summary'][0]
    entity_b_first_summary = item['entity_b_summary'][0]
    
    # Append the summaries to their respective lists
    entity_a_summaries.append(entity_a_first_summary)
    entity_b_summaries.append(entity_b_first_summary)

# Print the lists
print("Entity A Summaries:", entity_a_summaries)
print("Entity B Summaries:", entity_b_summaries)


Entity A Summaries: ['This all suite hotel is flawless. Around the holidays season, the hotel was well decorated for X-mas with lovely Palm trees and Christmas lights. This hotel is so close to the beach and is perfectly located for peace and quiet. The airport and Duval street is only a short shuttle away, but downtown is quite far away. Morden rooms with great size and a lot of storage space. The beds are super comfortable. The bathroom is also pretty large with a nice vanity area. The staff were even willing to help with last minute late checkout request. The restaurant provides great food, but it closes too early for us. The hotel actually has bikes to rent. Parking close-by can be very pricey per day. The elevators feel not safe.', "The hotel has an enjoyable ambience and is great value for money. This location is great if you're looking to access the great fun and action close-by, but with that comes hustle-and-bustle noise from the street the hotel is located on. The rustic room

# 실험 

In [21]:
end = len(entity_a_summaries)
CASPR_list = []

for i in range(0, end):
    # CASPR 점수 계산
    summary_A_B = entity_a_summaries[i]
    summary_B_A = entity_b_summaries[i]
    print(f"[{i}]")
    caspr_score = calculate_caspr(summary_A_B, summary_B_A)
    print(f"CASPR Score: {caspr_score}\n\n\n")
    
    CASPR_list.append(caspr_score)

[0]
Decomposed Sentences: ['This all suite hotel is flawless', ' Around the holidays season, the hotel was well decorated for X-mas with lovely Palm trees and Christmas lights', ' This hotel is so close to the beach and is perfectly located for peace and quiet', ' The airport and Duval street is only a short shuttle away, but downtown is quite far away', ' Morden rooms with great size and a lot of storage space', ' The beds are super comfortable', ' The bathroom is also pretty large with a nice vanity area', ' The staff were even willing to help with last minute late checkout request', ' The restaurant provides great food, but it closes too early for us', ' The hotel actually has bikes to rent', ' Parking close-by can be very pricey per day', ' The elevators feel not safe', '']

Sentences from Summary 1: ['This all suite hotel is flawless', ' Around the holidays season, the hotel was well decorated for X-mas with lovely Palm trees and Christmas lights', ' This hotel is so close to the 

In [22]:
CASPR_list

[100.0,
 100.0,
 100.0,
 78.57142857142857,
 87.5,
 100.0,
 100.0,
 100.0,
 100.0,
 88.23529411764706,
 100.0,
 88.23529411764706,
 94.11764705882352,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 92.85714285714286]

In [24]:
import scipy.stats as stats

# Function to compute 95% confidence interval
def compute_confidence_interval(data):
    mean = np.mean(data)
    sem = stats.sem(data) # Standard error of the mean
    ci = stats.t.interval(0.95, len(data)-1, loc=mean, scale=sem)
    return ci

# Compute 95% confidence interval
CASPR_ci = compute_confidence_interval(CASPR_list)

# Print the results
CASPR_ci

(93.57590214898522, 99.37577852328369)

In [27]:
import numpy as np

# Function to generate bootstrap samples and calculate the confidence interval
def bootstrap_confidence_interval(data, num_samples=10000, confidence_level=0.95):
    bootstrap_means = []
    n = len(data)
    
    # Generate bootstrap samples and calculate the mean of each sample
    for _ in range(num_samples):
        sample = np.random.choice(data, size=n, replace=True)
        bootstrap_means.append(np.mean(sample))
    
    # Calculate the confidence interval
    lower_percentile = (1.0 - confidence_level) / 2.0
    upper_percentile = 1.0 - lower_percentile
    ci_lower = np.percentile(bootstrap_means, lower_percentile * 100)
    ci_upper = np.percentile(bootstrap_means, upper_percentile * 100)
    
    return ci_lower, ci_upper

# Compute 95% bootstrap confidence interval
CASPR_ci_bootstrap = bootstrap_confidence_interval(CASPR_list)

# Print the results
CASPR_ci_bootstrap

(93.65546218487395, 98.82352941176471)

# 성능평가 

In [1]:
import pandas as pd 

system_file = '/root/system_hotel_result10.csv'
human_file = '/root/human_hotel_dataset10.csv'

system_df = pd.read_csv(system_file)
human_df = pd.read_csv(human_file)

def aspectList(df):
    unique_items = df['item_name'].unique()
    aspect_dict = {item:[] for item in unique_items}
    
    for index, row in df.iterrows():
        aspect_dict[row['item_name']].append(row['aspect'])
        
    for item, aspects in aspect_dict.items():
        print(f"{item}: {aspects}")
        
    return aspect_dict

def keyList(df):
    key_dict = {}
    for index, item_name in enumerate(df['item_name'].unique()):
        key_dict[index] = item_name
        
    return key_dict

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

ASmodel =  SentenceTransformer("jhgan/ko-sroberta-multitask")
def aspect_similar(my_aspect, system_aspect):
   embeddings = ASmodel.encode([my_aspect, system_aspect])
   
   my_embedding = embeddings[0]
   system_embedding = embeddings[1]
   
   similarity_score = cosine_similarity([my_embedding], [system_embedding])[0][0]
   
   return similarity_score

# 시스템/휴먼의 aspect 리스트 
system_aspect_list = aspectList(system_df)
human_aspect_list = aspectList(human_df)

system_key_list = keyList(system_df)
human_key_list = keyList(human_df)

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


제주신라호텔 (The Shilla Jeju): ['직원 서비스', '객실 상태', '위치 및 교통', '편의시설', '음식 품질', '재방문 의사', '호텔 분위기', '시설 만족도']
그랜드 조선 제주 (Grand Josun Jeju): ['직원 서비스', '객실 상태', '위치 및 교통', '편의시설', '음식 품질', '재방문 의사', '호텔 분위기', '시설 만족도']
도미인 서울 강남 (Dormy Inn SEOUL Gangnam): ['조식 퀄리티', '직원 서비스', '객실 청결 상태', '위치 및 주변 환경', '사우나 시설 및 이용 ', '재방문 의사', '친절한 응대', '만족스러운 휴식 제공']
글래드 마포 (GLAD MAPO): ['조식 퀄리티', '직원 서비스', '객실 청결 상태', '위치 및 주변 환경', '사우나 시설 및 이용 ', '재방문 의사', '친절한 응대', '만족스러운 휴식 제공']
호텔 피제이 명동 (Hotel PJ Myeongdong): ['위치', '주차 시설', '객실 상태', '직원 서비스', '시설 청결상태', '소음 및 방음', '편의 시설', '가격 대비 만족도']
나인트리 호텔 명동 (Nine Tree Hotel Myeong-dong): ['위치', '주차 시설', '객실 상태', '직원 서비스', '시설 청결상태', '소음 및 방음', '편의 시설', '가격 대비 만족도']
신라스테이 서부산 김해공항 (Shilla Stay Seobusan Gimhae Airport): ['직원 서비스', '객실 상태', '위치 및 교통', '청결 상태', '시설의 만족도', '화장실 상태', '불편함 및 불만해요']
센텀 프리미어 호텔 (Centum Premier Hotel (Korea Quality)): ['직원 서비스', '객실 상태', '위치 및 교통', '청결 상태', '시설의 만족도', '화장실 상태', '불편함 및 불만해요']
전주 시그니처 호텔 앤 레지던스 (Jeonju Signature Hotel & Res

In [14]:
from scipy import stats

# 95% 신뢰구간 계산
def calculate_confidence_interval(data, confidence=0.95):
    n = len(data)
    mean = np.mean(data)
    sem = stats.sem(data)
    interval = sem * stats.t.ppf((1 + confidence) / 2., n-1)
    return mean, mean - interval, mean + interval


# 서로 유사도가 높은 aspect끼리 요약 유사도 평가
def MaxAspectSimilar(system_df, human_df):
    total_data = []; 
    
    for n in range(len(human_key_list)): # 호텔마다 반복 
        rouge_1 = 0;  CASPR_Score_CI = ""
        
        if(human_key_list[n] == system_key_list[n]): # 아이템 이름이 같은 경우 
            
            for hum_aspect in human_aspect_list[human_key_list[n]]: # 해당 아이템 내 aspect들 중 하나 
                max_similarity = 0; CASPR_list = []
                
                for sys_aspect in system_aspect_list[system_key_list[n]]: # hum aspect와 가장 유사한 sys aspect 찾기 
                    similarity = aspect_similar(hum_aspect, sys_aspect)
                    if similarity > max_similarity:
                        max_similarity = similarity
                        max_sys_aspect = sys_aspect
                    
                    print(max_sys_aspect, ", ", max_similarity)
                    
                hum_summary = human_df[human_df['item_name'] == human_key_list[n]][human_df['aspect'] == hum_aspect]['summary'].values[0]
                
                sys_summary = system_df[system_df['item_name'] == system_key_list[n]][system_df['aspect'] == max_sys_aspect]['summary'].values[0]
                
                caspr_score = calculate_caspr(sys_summary, hum_summary)
                print("\n\n\n\nCASPR_SCORE: ", caspr_score, "\n\n\n\n")
                CASPR_list.append(caspr_score)
                
            
            # len_hum_aspect = len(human_aspect_list[human_key_list[n]])
            # CASPR_Score_CI = sum(CASPR_list) / len_hum_aspect
            print("\n\n\n\nitem_name: ", human_key_list[n], "CASPR_list: ", CASPR_list,"\n\n\n\n")
            
            #95% 신뢰구간 구하기 
            if CASPR_list:  # CASPR_list가 비어 있지 않은 경우에만 계산
                mean_score, lower_bound, upper_bound = calculate_confidence_interval(CASPR_list)
                CASPR_Score_CI = f"{mean_score:.2f} ({lower_bound:.2f}, {upper_bound:.2f})"
            else:
                CASPR_Score_CI = "N/A"
            
            data = [human_key_list[n], CASPR_Score_CI] #그냥 bertScore
            total_data.append(data)
    
    return total_data

In [15]:
total_data = MaxAspectSimilar(system_df, human_df)

total_df = pd.DataFrame(total_data, columns=['item_name', 'CASPR_CI'])

total_df.head(10)

직원 서비스 ,  1.0000002
직원 서비스 ,  1.0000002
직원 서비스 ,  1.0000002
직원 서비스 ,  1.0000002
직원 서비스 ,  1.0000002
직원 서비스 ,  1.0000002
직원 서비스 ,  1.0000002
직원 서비스 ,  1.0000002
Decomposed Sentences: ['호텔 직원은 친절하고 고객 대접이 좋다', ' 특급 호텔 서비스에 대한 인상은 크지 않지만, 친절함은 기본 수준이며 특별한 서비스에 대한 부족함을 느낀 후기가 있다', ' 신라호텔 직원은 친절함이 뛰어나다는 긍정적인 의견과 만족스러운 서비스를 경험했다는 후기가 있다', '']

Sentences from Summary 1: ['호텔 직원은 친절하고 고객 대접이 좋다', ' 특급 호텔 서비스에 대한 인상은 크지 않지만, 친절함은 기본 수준이며 특별한 서비스에 대한 부족함을 느낀 후기가 있다', ' 신라호텔 직원은 친절함이 뛰어나다는 긍정적인 의견과 만족스러운 서비스를 경험했다는 후기가 있다', '']
Decomposed Sentences: ['종종 객실 업그레이드를 해준다', ' 직원들의 응대가 아주 친절하다', '']

Sentences from Summary 2: ['종종 객실 업그레이드를 해준다', ' 직원들의 응대가 아주 친절하다', '']



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:32: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Comparing '호텔 직원은 친절하고 고객 대접이 좋다' and '종종 객실 업그레이드를 해준다':
 - Result 1: [{'label': 'CONTRADICTION', 'score': 0.3498535454273224}]
 - Result 2: [{'label': 'CONTRADICTION', 'score': 0.3919777572154999}]

Relationship between '호텔 직원은 친절하고 고객 대접이 좋다' and '종종 객실 업그레이드를 해준다': contradiction

Comparing '호텔 직원은 친절하고 고객 대접이 좋다' and ' 직원들의 응대가 아주 친절하다':
 - Result 1: [{'label': 'ENTAILMENT', 'score': 0.44715556502342224}]
 - Result 2: [{'label': 'NEUTRAL', 'score': 0.438002347946167}]

Relationship between '호텔 직원은 친절하고 고객 대접이 좋다' and ' 직원들의 응대가 아주 친절하다': entailment

s1 : 호텔 직원은 친절하고 고객 대접이 좋다

Contradictions: 1, Entailments: 1, Neutrals: 0
Comparing ' 특급 호텔 서비스에 대한 인상은 크지 않지만, 친절함은 기본 수준이며 특별한 서비스에 대한 부족함을 느낀 후기가 있다' and '종종 객실 업그레이드를 해준다':
 - Result 1: [{'label': 'ENTAILMENT', 'score': 0.44243815541267395}]
 - Result 2: [{'label': 'ENTAILMENT', 'score': 0.3998342752456665}]

Relationship between ' 특급 호텔 서비스에 대한 인상은 크지 않지만, 친절함은 기본 수준이며 특별한 서비스에 대한 부족함을 느낀 후기가 있다' and '종종 객실 업그레이드를 해준다': entailment

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:32: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Comparing '호텔 방은 낡은 느낌이 있지만 관리가 잘 되어 클래식한 매력을 느낄 수 있다는 후기와 낡아 보이는 가구와 화장실 등에 대한 부정적 의견이 있다' and '낡은 인테리어 디자인이고, 노후화되었다':
 - Result 1: [{'label': 'ENTAILMENT', 'score': 0.47847411036491394}]
 - Result 2: [{'label': 'ENTAILMENT', 'score': 0.4670230746269226}]

Relationship between '호텔 방은 낡은 느낌이 있지만 관리가 잘 되어 클래식한 매력을 느낄 수 있다는 후기와 낡아 보이는 가구와 화장실 등에 대한 부정적 의견이 있다' and '낡은 인테리어 디자인이고, 노후화되었다': entailment

Comparing '호텔 방은 낡은 느낌이 있지만 관리가 잘 되어 클래식한 매력을 느낄 수 있다는 후기와 낡아 보이는 가구와 화장실 등에 대한 부정적 의견이 있다' and ' 칫솔이 비치되어 있지 않다':
 - Result 1: [{'label': 'ENTAILMENT', 'score': 0.4314993619918823}]
 - Result 2: [{'label': 'ENTAILMENT', 'score': 0.46346643567085266}]

Relationship between '호텔 방은 낡은 느낌이 있지만 관리가 잘 되어 클래식한 매력을 느낄 수 있다는 후기와 낡아 보이는 가구와 화장실 등에 대한 부정적 의견이 있다' and ' 칫솔이 비치되어 있지 않다': entailment

s1 : 호텔 방은 낡은 느낌이 있지만 관리가 잘 되어 클래식한 매력을 느낄 수 있다는 후기와 낡아 보이는 가구와 화장실 등에 대한 부정적 의견이 있다

Contradictions: 0, Entailments: 2, Neutrals: 0
Comparing ' 객실 상태가 깨끗하고 화장실이 넓다는 긍정적인 평도 있다' and '낡은 인테리어 디자인이고, 노후화되었다':

...

Comparing '호텔은 좋은 위치와 외관을 가지고 있으나 다음에는 다른 숙소를 고려하고 싶다' and '근처에 음식점이 많다':
 - Result 1: [{'label': 'NEUTRAL', 'score': 0.3482859134674072}]
 - Result 2: [{'label': 'ENTAILMENT', 'score': 0.3714214265346527}]

Relationship between '호텔은 좋은 위치와 외관을 가지고 있으나 다음에는 다른 숙소를 고려하고 싶다' and '근처에 음식점이 많다': entailment

Comparing '호텔은 좋은 위치와 외관을 가지고 있으나 다음에는 다른 숙소를 고려하고 싶다' and ' 건물을 상가와 예식장을 함께 사용한다':
 - Result 1: [{'label': 'ENTAILMENT', 'score': 0.35396701097488403}]
 - Result 2: [{'label': 'ENTAILMENT', 'score': 0.3939383327960968}]

Relationship between '호텔은 좋은 위치와 외관을 가지고 있으나 다음에는 다른 숙소를 고려하고 싶다' and ' 건물을 상가와 예식장을 함께 사용한다': entailment

s1 : 호텔은 좋은 위치와 외관을 가지고 있으나 다음에는 다른 숙소를 고려하고 싶다

Contradictions: 0, Entailments: 2, Neutrals: 0
Comparing ' 주변 음식점과 주차장이 편리하다고 언급하였다' and '근처에 음식점이 많다':
 - Result 1: [{'label': 'CONTRADICTION', 'score': 0.3570898175239563}]
 - Result 2: [{'label': 'ENTAILMENT', 'score': 0.36791959404945374}]

Relationship between ' 주변 음식점과 주차장이 편리하다고 언급하였다' and '근처에 음식점이 많다': neut

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:32: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Comparing '가격 대비 룸 컨디션의 저하로 인해 다른 모텔보다 못하다고 언급하였으며, 호텔 수준에 미치지 못하는 품질에 대한 불만을 표현하였다' and '저렴한 가격에 가성비가 있다':
 - Result 1: [{'label': 'ENTAILMENT', 'score': 0.40998947620391846}]
 - Result 2: [{'label': 'ENTAILMENT', 'score': 0.40869274735450745}]

Relationship between '가격 대비 룸 컨디션의 저하로 인해 다른 모텔보다 못하다고 언급하였으며, 호텔 수준에 미치지 못하는 품질에 대한 불만을 표현하였다' and '저렴한 가격에 가성비가 있다': entailment

s1 : 가격 대비 룸 컨디션의 저하로 인해 다른 모텔보다 못하다고 언급하였으며, 호텔 수준에 미치지 못하는 품질에 대한 불만을 표현하였다

Contradictions: 0, Entailments: 1, Neutrals: 0
Decomposed Sentences: ['저렴한 가격에 가성비가 있다', '']

Sentences from Summary 1: ['저렴한 가격에 가성비가 있다', '']
Decomposed Sentences: ['가격 대비 룸 컨디션의 저하로 인해 다른 모텔보다 못하다고 언급하였으며, 호텔 수준에 미치지 못하는 품질에 대한 불만을 표현하였다', '']

Sentences from Summary 2: ['가격 대비 룸 컨디션의 저하로 인해 다른 모텔보다 못하다고 언급하였으며, 호텔 수준에 미치지 못하는 품질에 대한 불만을 표현하였다', '']

Comparing '저렴한 가격에 가성비가 있다' and '가격 대비 룸 컨디션의 저하로 인해 다른 모텔보다 못하다고 언급하였으며, 호텔 수준에 미치지 못하는 품질에 대한 불만을 표현하였다':
 - Result 1: [{'label': 'ENTAILMENT', 'score': 0.4231548309326172}]
 - Result 2

/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:263: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,item_name,CASPR_CI
0,제주신라호텔 (The Shilla Jeju),"0.00 (nan, nan)"
1,그랜드 조선 제주 (Grand Josun Jeju),"0.00 (nan, nan)"
2,도미인 서울 강남 (Dormy Inn SEOUL Gangnam),"33.33 (nan, nan)"
3,글래드 마포 (GLAD MAPO),"0.00 (nan, nan)"
4,호텔 피제이 명동 (Hotel PJ Myeongdong),"25.00 (nan, nan)"
5,나인트리 호텔 명동 (Nine Tree Hotel Myeong-dong),"0.00 (nan, nan)"
6,신라스테이 서부산 김해공항 (Shilla Stay Seobusan Gimhae Ai...,"0.00 (nan, nan)"
7,센텀 프리미어 호텔 (Centum Premier Hotel (Korea Quality)),"0.00 (nan, nan)"
8,전주 시그니처 호텔 앤 레지던스 (Jeonju Signature Hotel & Re...,"0.00 (nan, nan)"
9,라한호텔 전주 (Lahan Hotel Jeonju),"0.00 (nan, nan)"


In [13]:
total_df.head(10)

,item_name,CASPR_CI
0,제주신라호텔 (The Shilla Jeju),0.000000
1,그랜드 조선 제주 (Grand Josun Jeju),0.000000
2,도미인 서울 강남 (Dormy Inn SEOUL Gangnam),6.666667
3,글래드 마포 (GLAD MAPO),0.000000
4,호텔 피제이 명동 (Hotel PJ Myeongdong),5.000000
5,나인트리 호텔 명동 (Nine Tree Hotel Myeong-dong),0.000000
6,신라스테이 서부산 김해공항 (Shilla Stay Seobusan Gimhae Ai...,0.000000
7,센텀 프리미어 호텔 (Centum Premier Hotel (Korea Quality)),0.000000
8,전주 시그니처 호텔 앤 레지던스 (Jeonju Signature Hotel & Re...,0.000000
9,라한호텔 전주 (Lahan Hotel Jeonju),0.000000


In [10]:
total_df.to_csv('/root/hotel_CASPR_CI.csv', index=False)

In [16]:
caspr_df = pd.read_csv('/root/hotel_CASPR_CI.csv')

In [17]:
caspr_df.head(10)

,item_name,CASPR_CI
0,제주신라호텔 (The Shilla Jeju),"0.00 (nan, nan)"
1,그랜드 조선 제주 (Grand Josun Jeju),"0.00 (nan, nan)"
2,도미인 서울 강남 (Dormy Inn SEOUL Gangnam),"33.33 (nan, nan)"
3,글래드 마포 (GLAD MAPO),"0.00 (nan, nan)"
4,호텔 피제이 명동 (Hotel PJ Myeongdong),"25.00 (nan, nan)"
5,나인트리 호텔 명동 (Nine Tree Hotel Myeong-dong),"0.00 (nan, nan)"
6,신라스테이 서부산 김해공항 (Shilla Stay Seobusan Gimhae Ai...,"0.00 (nan, nan)"
7,센텀 프리미어 호텔 (Centum Premier Hotel (Korea Quality)),"0.00 (nan, nan)"
8,전주 시그니처 호텔 앤 레지던스 (Jeonju Signature Hotel & Re...,"0.00 (nan, nan)"
9,라한호텔 전주 (Lahan Hotel Jeonju),"0.00 (nan, nan)"
